# 1. Import dependencies

In [5]:
pip install stable-baselines3[extra]

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pyglet

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Environments

In [8]:
environment_name = 'CartPole-v0'
env = gym.make(environment_name)

In [9]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset() # Observation
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample() # 0 or 1 (Discrete)
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode: {} Score: {}'.format(episode, score))

Episode: 1 Score: 23.0
Episode: 2 Score: 15.0
Episode: 3 Score: 24.0
Episode: 4 Score: 19.0
Episode: 5 Score: 42.0


In [10]:
env.close()

## Understanding The Environment
[https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py)

In [11]:
env.action_space

Discrete(2)

In [12]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

# 3. Training

In [13]:
log_path = os.path.join('training', 'logs')

In [14]:
log_path

'training/logs'

In [15]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [16]:
PPO??

Init signature:
PPO(
    policy: Union[str, Type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gym.core.Env, stable_baselines3.common.vec_env.base_vec_env.VecEnv, str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    target_kl: Union[float, NoneType] = None,
    tensorboard_log: Union[str, NoneType] = None,
    create_eval_env: bool = False,
    policy_kwargs: Union[Dict[str, Any], NoneType] = None,
    verbose: int = 0,
    seed: Union[int, NoneType] = None,
    device: Union[torch.device, str] = 'auto',
    _init_setup_model: bool = 

In [17]:
model.learn(total_timesteps=20000) # self driving app needs more 'total_timesteps' 300,000 or 400,000

Logging to training/logs/PPO_6
-----------------------------
| time/              |      |
|    fps             | 1258 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 931         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008197685 |
|    clip_fraction        | 0.0846      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | 0.00175     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.26        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0122     |
|    value_loss           | 52.1        |
-----------------------------------------
---

In [18]:
PPO_PATH = os.path.join('training', 'saved_models', 'PPO_Model_Cartpole')

In [19]:
model.save(PPO_PATH)

In [20]:
del model

In [21]:
model = PPO.load(PPO_PATH, env=env)

# 5. Evaluation

In [22]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/justin/github/rl_3hour_course/rl_3hours/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [23]:
env.close()

# 6. Test Model

In [24]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset() # Observation
    done = False
    score = 0
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode: {} Score: {}'.format(episode, score))

Episode: 1 Score: [173.]
Episode: 2 Score: [200.]
Episode: 3 Score: [200.]
Episode: 4 Score: [200.]
Episode: 5 Score: [200.]


In [25]:
env.close()

In [26]:
obs = env.reset()
action, _ = model.predict(obs)

In [27]:
obs

array([[-0.04619795, -0.04635945,  0.02371588, -0.00227657]],
      dtype=float32)

In [28]:
env.action_space.sample()

0

In [29]:
env.step(action)

(array([[-0.04712514,  0.1484145 ,  0.02367035, -0.28738353]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{}])

# 7. Viewing Logs in Tensorboard

In [30]:
training_log_path = os.path.join(log_path, 'PPO_2')

### To run tensorboard
``` bash
cd LOG_PATH
tensorboard --logdir=.

```

# 8. Adding a callback to the Stage

In [31]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [32]:
save_path = os.path.join('training', 'saved_models')

In [33]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                            callback_on_new_best=stop_callback, 
                            eval_freq=10000, 
                            best_model_save_path=save_path, 
                            verbose=1)

In [34]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [36]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to training/logs/PPO_7
-----------------------------
| time/              |      |
|    fps             | 868  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 614         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009150157 |
|    clip_fraction        | 0.101       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00461     |
|    learning_rate        | 0.0003      |
|    loss                 | 5.44        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0158     |
|    value_loss           | 54.8        |
-----------------------------------------
---

/home/justin/github/rl_3hour_course/rl_3hours/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 200         |
|    mean_reward          | 200         |
| time/                   |             |
|    total timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.009070033 |
|    clip_fraction        | 0.0604      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.609      |
|    explained_variance   | 0.253       |
|    learning_rate        | 0.0003      |
|    loss                 | 24          |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0169     |
|    value_loss           | 68          |
-----------------------------------------
New best mean reward!
Stopping training because the mean reward 200.00  is above the threshold 200


# 9. Change Policies

In [38]:
net_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [39]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cuda device


In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

### Try DQN

In [41]:
from stable_baselines3 import DQN

In [42]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [ ]:
model.learn(total_timesteps=20000)

In [45]:
DQN_PATH = os.path.join('training', 'saved_models', 'DQN_Model_Cartpole')

In [46]:
model.save(DQN_PATH)

In [47]:
model = DQN.load(DQN_PATH)